In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
coefMat = pd.read_csv('coefficient matrix.csv')
data = pd.read_csv('1718hourlyReturn_withZEC.csv')
macro = pd.read_csv('macro1718_return.csv')

In [4]:
# set the time frame for analysis

timeAna = data.time[48:]
timeAna.index =range(len(timeAna))
start, end = timeAna.index[timeAna=='12/15/2017 0:00'], timeAna.index[timeAna=='12/22/2017 0:00']
start, end = int(start.values), int(end.values)
timeAna = timeAna[start:end]

# Moving Network

In [14]:
for time in timeAna: 
    t = int(timeAna.index[timeAna == time].values)
    hourlyMat = coefMat.iloc[:,(14*t):14*(t+1)]
    
    # create the nodes
    G = nx.DiGraph()
    G.add_nodes_from(data.columns[1:], category = 'coin')
    G.add_nodes_from(macro.columns[1:], category = 'macro')
    
    # create the edges
    nodes = list(G.nodes())
    for i in range(hourlyMat.shape[0]):
        for j in range(hourlyMat.shape[1]):
            if hourlyMat.iloc[i,j] != 0:
                G.add_edges_from([(nodes[j], nodes[i])], weight = abs(hourlyMat.iloc[i,j]), direction = np.sign(hourlyMat.iloc[i,j]))

    # distinguish the nodes between coins and macro variables            
    col = []
    for x in nx.get_node_attributes(G,'category').values():
        if x =='coin':
            col.append('orange')
        else:
            col.append('lightblue')
    
    # generate a circular network graph
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    pos = nx.circular_layout(G)
    count = np.count_nonzero(hourlyMat, axis=0)
    plt.figure(figsize= (5,5)) # 20,20
    plt.title(f'Date: {time}', fontsize = 12, loc = 'right')
    nx.draw(G, pos, node_size = count*100, node_color = col, edgelist=edges, width=weights, with_labels = True,
           alpha = 0.8, arrowstyle = '->', font_size = 12) 
#     plt.savefig("graph{:04d}.png".format(t+1), format="PNG")

## Creating a gif file for the moving network

In [16]:
# create respective folders 'png' and 'gif' and move the individual network graphics into the 'png' folder.
# after running this code, the created gif file will be saved in the 'gif' folder
import os
import imageio

png_dir = 'C:/Users/Seokhee2/Documents/HU/WiSe1920/Digital Economy and Data Analysis/project/png'
images = []
for file_name in os.listdir(png_dir):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave('C:/Users/Seokhee2/Documents/HU/WiSe1920/Digital Economy and Data Analysis/project/gif/moving_network.gif', images)